<a href="https://colab.research.google.com/github/Saucymukhim/Chisur-Academy-Test-01/blob/main/T24CS015_Codefile_2nd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**EFFICIENTNETV2 + TRANSFORMER HEAD**

SECTION 1: SETUP & INSTALLATION

In [1]:
# Import Google Drive library
from google.colab import drive

In [2]:
# Mount Google Drive to access files
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd /content/drive/MyDrive/SaucyDataset

/content/drive/MyDrive/SaucyDataset


In [4]:
# Install required packages for this project
!pip install --upgrade -q timm scikit-learn seaborn
!pip install optuna
!pip install torchmetrics
!pip install opencv-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.9/413.9 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 25.7 MB/s eta 0:00:00


In [12]:
!pip install torchmetrics
!pip install opencv-python


In [5]:
# Import necessary libraries for deep learning, data manipulation, and visualization.
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, Subset
from torchvision import datasets, transforms
import timm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (confusion_matrix, classification_report,
                            accuracy_score, roc_auc_score, roc_curve,
                            precision_recall_curve, average_precision_score,
                            precision_recall_fscore_support)
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
import pandas as pd
import cv2
import random
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# For hyperparameter tuning
import optuna

# For model calibration
from torchmetrics import CalibrationError

print("✅ All packages installed and imported successfully!")

✅ All packages installed and imported successfully!


SECTION 2: CONFIGURATION

In [17]:
# Central configuration class to manage all hyperparameters, paths, and settings.
class Config:
    """Configuration class for the bone fracture detection project."""

    def __init__(self):
        # Data paths
        self.DATA_PATH = "/content/drive/MyDrive/SaucyDataset"
        self.OUTPUT_DIR = "/content/drive/MyDrive/SaucyDataset_Output"

        # Create output directory if it doesn't exist
        os.makedirs(self.OUTPUT_DIR, exist_ok=True)

        # Model & Training Parameters
        self.MODEL_NAME = 'tf_efficientnetv2_s.in1k'
        self.IMAGE_SIZE = 256
        self.BATCH_SIZE = 8
        self.EPOCHS = 30
        self.LEARNING_RATE = 1e-4
        self.NUM_CLASSES = 2  # 'fractured' and 'not_fractured'

        # Transformer Head Parameters
        self.D_MODEL = 256
        self.NHEAD = 8
        self.NUM_ENCODER_LAYERS = 4
        self.DIM_FEEDFORWARD = 2048
        self.DROPOUT = 0.1

        # Early stopping parameters
        self.EARLY_STOPPING_PATIENCE = 5

        # K-Fold CV parameters
        self.K_FOLDS = 2

        # For reproducibility
        self.SEED = 42

        # Device configuration
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Create a global configuration instance to be used throughout the script
CFG = Config()

print(f"✅ Configuration loaded. Device set to: {CFG.device}")


✅ Configuration loaded. Device set to: cuda:0


SECTION 3: DATA HANDLING & TRANSFORMATIONS

In [7]:
# Classes responsible for loading data, applying transformations, and managing datasets.
class DataTransforms:
    """Class to handle data transformations for different phases (train, val, test, tta)."""

    @staticmethod
    def get_transforms(image_size, phase='train'):
        """Get data transforms based on phase."""
        if phase == 'train':
            return transforms.Compose([
                transforms.RandomResizedCrop(image_size),
                transforms.RandomHorizontalFlip(),
                transforms.RandomRotation(10),
                transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
        elif phase in ['val', 'test']:
            return transforms.Compose([
                transforms.Resize(256),
                transforms.CenterCrop(image_size),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
        elif phase == 'tta':  # Test Time Augmentation
            return [
                transforms.Compose([transforms.Resize(256), transforms.CenterCrop(image_size), transforms.ToTensor(), transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]),
                transforms.Compose([transforms.Resize(256), transforms.CenterCrop(image_size), transforms.RandomHorizontalFlip(p=1.0), transforms.ToTensor(), transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]),
                transforms.Compose([transforms.Resize(256), transforms.CenterCrop(image_size), transforms.RandomRotation(10), transforms.ToTensor(), transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
            ]
        else:
            raise ValueError(f"Unknown phase: {phase}")

class DatasetManager:
    """Class to handle dataset operations like loading, splitting, and creating dataloaders."""

    @staticmethod
    def load_dataset(data_path):
        """Load X-ray dataset from folders."""
        if not os.path.exists(data_path):
            raise FileNotFoundError(f"Data path not found: {data_path}. Please check your CFG.DATA_PATH.")
        print(f"Loading dataset from {data_path}")
        return data_path

    @staticmethod
    def create_dataset_splits(data_path, image_size):
        """Create dataset partitions for train, validation, and test."""
        data_transforms = {
            'train': DataTransforms.get_transforms(image_size, 'train'),
            'val': DataTransforms.get_transforms(image_size, 'val'),
            'test': DataTransforms.get_transforms(image_size, 'test')
        }
        image_datasets = {x: datasets.ImageFolder(os.path.join(data_path, x), data_transforms[x])
                         for x in ['train', 'val', 'test']}
        return image_datasets

    @staticmethod
    def create_dataloaders(image_datasets, batch_size):
        """Create batched, shuffled data loaders."""
        dataloaders = {x: DataLoader(image_datasets[x], batch_size=batch_size,
                                    shuffle=(x=='train'), num_workers=2)
                      for x in ['train', 'val', 'test']}
        dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}
        class_names = image_datasets['train'].classes

        print(f"Datasets loaded. Class names: {class_names}")
        for split, size in dataset_sizes.items():
            print(f"   - {split.capitalize()} set size: {size} images")
        return dataloaders, dataset_sizes, class_names

    @staticmethod
    def prepare_metadata(image_datasets):
        """Prepare metadata including file paths and labels for multi-center validation."""
        metadata = {}
        for split in ['train', 'val', 'test']:
            dataset = image_datasets[split]
            samples = dataset.samples
            class_to_idx = dataset.class_to_idx
            split_metadata = []
            for path, label in samples:
                filename = os.path.basename(path)
                class_name = [k for k, v in class_to_idx.items() if v == label][0]
                site_id = f"site_{random.randint(1, 5)}" # Simulated site ID
                split_metadata.append({'filepath': path, 'filename': filename, 'label': label, 'class_name': class_name, 'site_id': site_id})
            metadata[split] = split_metadata
        with open(os.path.join(CFG.OUTPUT_DIR, 'metadata.json'), 'w') as f:
            json.dump(metadata, f, indent=2)
        print(f"Metadata prepared and saved to {os.path.join(CFG.OUTPUT_DIR, 'metadata.json')}")
        return metadata

print("✅ Data Handling classes defined.")

✅ Data Handling classes defined.


SECTION 4: MODEL ARCHITECTURE

In [8]:
# Definition of the Hybrid CNN-Transformer model.
class HybridModel(nn.Module):
    """EfficientNetV2 backbone + Transformer head."""

    def __init__(self, cnn_model_name, num_classes, d_model, nhead, num_encoder_layers, dim_feedforward, dropout):
        super(HybridModel, self).__init__()

        # 1. CNN Backbone (EfficientNetV2)
        self.backbone = timm.create_model(cnn_model_name, pretrained=True, features_only=True, out_indices=[-1])

        # 2. Transformer Head
        self.cls_token = nn.Parameter(torch.zeros(1, 1, d_model))
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward, dropout=dropout, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_encoder_layers)

        # 3. Classification Head
        self.classifier = nn.Linear(d_model, num_classes)

    def forward(self, x):
        """Define the forward pass of the model."""
        features = self.backbone(x)[0]  # Shape: (batch_size, d_model, H, W)
        batch_size = features.shape[0]
        seq = features.flatten(2).permute(0, 2, 1)  # (batch_size, seq_len, d_model)
        cls_tokens = self.cls_token.expand(batch_size, -1, -1)
        seq = torch.cat((cls_tokens, seq), dim=1)
        transformer_output = self.transformer_encoder(seq)
        cls_output = transformer_output[:, 0, :]
        logits = self.classifier(cls_output)
        return logits

print("✅ HybridModel architecture defined.")

✅ HybridModel architecture defined.


SECTION 5: MODEL UTILITIES

In [9]:
# Helper functions for model operations like freezing layers, setting up optimizers, etc.
class ModelUtils:
    """Utility functions for model operations."""

    @staticmethod
    def freeze_backbone(model):
        """Freeze backbone parameters."""
        for param in model.backbone.parameters(): param.requires_grad = False
        print("Backbone frozen")

    @staticmethod
    def unfreeze_backbone(model):
        """Unfreeze backbone parameters."""
        for param in model.backbone.parameters(): param.requires_grad = True
        print("Backbone unfrozen")

    @staticmethod
    def initialize_weights(model):
        """Apply model initialization."""
        for m in model.modules():
            if isinstance(m, nn.Linear): nn.init.xavier_uniform_(m.weight); nn.init.constant_(m.bias, 0) if m.bias is not None else None
            elif isinstance(m, nn.LayerNorm): nn.init.constant_(m.bias, 0); nn.init.constant_(m.weight, 1.0)
        print("Weights initialized")

    @staticmethod
    def get_loss_function(loss_type='cross_entropy'):
        """Define Cross-entropy / focal loss."""
        return nn.CrossEntropyLoss()

    @staticmethod
    def get_optimizer(model, learning_rate, weight_decay=1e-5):
        """AdamW optimizer settings."""
        return optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    @staticmethod
    def get_lr_scheduler(optimizer, scheduler_type='cosine', epochs=8):
        """Cosine annealing or ReduceLROnPlateau."""
        if scheduler_type == 'cosine': return optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs, eta_min=1e-6)
        elif scheduler_type == 'reduce_on_plateau': return optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3)
        else: return optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

    @staticmethod
    def save_checkpoint(model, optimizer, epoch, val_acc, path):
        """Save best-performing model."""
        torch.save({'epoch': epoch, 'model_state_dict': model.state_dict(), 'optimizer_state_dict': optimizer.state_dict(), 'val_acc': val_acc}, path)
        print(f"Checkpoint saved to {path}")

    @staticmethod
    def load_best_model(model, checkpoint_path):
        """Load trained checkpoint."""
        checkpoint = torch.load(checkpoint_path); model.load_state_dict(checkpoint['model_state_dict'])
        print(f"Model loaded from {checkpoint_path}"); return model

print("✅ Model utility functions defined.")

✅ Model utility functions defined.


SECTION 6: TRAINING & EVALUATION LOGIC

In [18]:
class TrainingUtils:
    """Utility functions for training and evaluation."""

    @staticmethod
    def train_one_epoch(model, dataloader, criterion, optimizer, device):
        """Train model for a single epoch."""
        model.train()
        running_loss, correct_predictions, total_samples = 0.0, 0, 0
        progress_bar = tqdm(dataloader, desc="Training", unit="batch")

        for inputs, labels in progress_bar:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct_predictions += torch.sum(preds == labels.data)
            total_samples += labels.size(0)

            progress_bar.set_postfix(
                loss=loss.item(),
                acc=f"{(correct_predictions.double() / total_samples):.4f}"
            )

        epoch_loss = running_loss / total_samples
        epoch_acc = correct_predictions.double() / total_samples
        return epoch_loss, epoch_acc.item()

    @staticmethod
    def evaluate(model, dataloader, criterion, device, split='Validation'):
        """Evaluate model for one epoch on a given split."""
        model.eval()
        running_loss, all_preds, all_labels, all_probs = 0.0, [], [], []
        progress_bar = tqdm(dataloader, desc=split, unit="batch")

        with torch.no_grad():
            for inputs, labels in progress_bar:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                running_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                probs = torch.softmax(outputs, dim=1)[:, 1]

                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
                all_probs.extend(probs.cpu().numpy())

        total_samples = len(all_labels)
        epoch_loss = running_loss / total_samples
        epoch_acc = accuracy_score(all_labels, all_preds)

        return epoch_loss, epoch_acc, all_labels, all_preds, all_probs

    @staticmethod
    def test_inference(model, test_loader, criterion, device):
        """Run model on the test set and report metrics."""
        model.eval()
        test_loss, test_acc, test_labels, test_preds, test_probs = TrainingUtils.evaluate(
            model, test_loader, criterion, device, split='Test'
        )
        print(f"Test Accuracy: {test_acc:.4f}")
        return test_loss, test_acc, test_labels, test_preds, test_probs


print("✅ Training and evaluation logic defined.")


✅ Training and evaluation logic defined.


SECTION 7: METRICS & VISUALIZATION

In [13]:
class MetricsAndVisualization:
    """Class for computing metrics and creating visualizations."""

    @staticmethod
    def compute_metrics(labels, preds, probs):
        """Compute Accuracy, Precision, Recall, F1, and AUC."""
        # No artificial accuracy cap or label flipping
        accuracy = accuracy_score(labels, preds)
        precision, recall, f1, _ = precision_recall_fscore_support(
            labels, preds, average='binary'
        )
        auc = roc_auc_score(labels, probs)

        return {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'auc': auc
        }

    @staticmethod
    def plot_curves(history, test_acc=None, test_auc=None):
        """Plot training/validation accuracy and loss curves."""
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 6))

        ax1.plot(history['train_acc'], label='Train Accuracy', color='#1f77b4',
                 marker='o', linestyle='-', lw=2)
        ax1.plot(history['val_acc'], label='Validation Accuracy', color='#ff7f0e',
                 marker='o', linestyle='-', lw=2)
        if test_acc is not None:
            ax1.axhline(
                y=test_acc,
                color='#2ca02c',
                linestyle='--',
                label=f'Test Accuracy: {test_acc:.4f}',
                lw=2
            )
        ax1.set_title('Model Accuracy Over Epochs', fontsize=16, fontweight='bold')
        ax1.set_xlabel('Epoch', fontsize=12)
        ax1.set_ylabel('Accuracy', fontsize=12)
        ax1.legend(fontsize=10)
        ax1.grid(True, linestyle='--', alpha=0.6)

        ax2.plot(history['train_loss'], label='Train Loss', color='#1f77b4',
                 marker='o', linestyle='-', lw=2)
        ax2.plot(history['val_loss'], label='Validation Loss', color='#ff7f0e',
                 marker='o', linestyle='-', lw=2)
        ax2.set_title('Model Loss Over Epochs', fontsize=16, fontweight='bold')
        ax2.set_xlabel('Epoch', fontsize=12)
        ax2.set_ylabel('Loss', fontsize=12)
        ax2.legend(fontsize=10)
        ax2.grid(True, linestyle='--', alpha=0.6)

        plt.tight_layout()
        plt.savefig(os.path.join(CFG.OUTPUT_DIR, 'training_curves.png'))
        plt.show()

    @staticmethod
    def plot_confusion_matrix(labels, preds, class_names):
        """Visualize confusion matrix."""
        cm = confusion_matrix(labels, preds)
        plt.figure(figsize=(8, 6))
        sns.heatmap(
            cm,
            annot=True,
            fmt='d',
            cmap='Blues',
            xticklabels=class_names,
            yticklabels=class_names
        )
        plt.xlabel('Predicted Label')
        plt.ylabel('True Label')
        plt.title('Confusion Matrix for Test Set')
        plt.savefig(os.path.join(CFG.OUTPUT_DIR, 'confusion_matrix.png'))
        plt.show()

    @staticmethod
    def analyze_predictions(model, dataloader, class_names, device, num_examples=5):
        """Visualize correct & misclassified image examples."""
        model.eval()
        all_preds, all_labels, all_probs, all_images = [], [], [], []

        with torch.no_grad():
            for inputs, labels in dataloader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                probs = torch.softmax(outputs, dim=1)
                _, preds = torch.max(outputs, 1)

                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
                all_probs.extend(probs.cpu().numpy())
                all_images.extend(inputs.cpu())

        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)
        all_probs = np.array(all_probs)

        # No accuracy cap or flipping here either
        correct_indices = np.where(all_preds == all_labels)[0]
        incorrect_indices = np.where(all_preds != all_labels)[0]

        print(f"\nDisplaying {min(num_examples, len(correct_indices))} correct predictions")
        for i, idx in enumerate(
            np.random.choice(correct_indices, min(num_examples, len(correct_indices)), replace=False)
        ):
            img = all_images[idx].permute(1, 2, 0).numpy()
            img = np.clip(
                img * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406]),
                0, 1
            )
            true_label = class_names[all_labels[idx]]
            pred_label = class_names[all_preds[idx]]
            confidence = all_probs[idx, all_preds[idx]]

            plt.figure(figsize=(4, 4))
            plt.imshow(img)
            plt.title(f"True: {true_label}\nPred: {pred_label} ({confidence:.2f})",
                      color='green')
            plt.axis('off')
            plt.savefig(os.path.join(CFG.OUTPUT_DIR, f'correct_pred_{i}.png'))
            plt.show()

        print(f"\nDisplaying {min(num_examples, len(incorrect_indices))} incorrect predictions")
        for i, idx in enumerate(
            np.random.choice(incorrect_indices, min(num_examples, len(incorrect_indices)), replace=False)
        ):
            img = all_images[idx].permute(1, 2, 0).numpy()
            img = np.clip(
                img * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406]),
                0, 1
            )
            true_label = class_names[all_labels[idx]]
            pred_label = class_names[all_preds[idx]]
            confidence = all_probs[idx, all_preds[idx]]

            plt.figure(figsize=(4, 4))
            plt.imshow(img)
            plt.title(f"True: {true_label}\nPred: {pred_label} ({confidence:.2f})",
                      color='red')
            plt.axis('off')
            plt.savefig(os.path.join(CFG.OUTPUT_DIR, f'incorrect_pred_{i}.png'))
            plt.show()

print("✅ Metrics and visualization functions defined.")


✅ Metrics and visualization functions defined.


SECTION 8: GRAD-CAM IMPLEMENTATION

In [14]:
class GradCAM:
    """
    Implements Gradient-weighted Class Activation Mapping (Grad-CAM).

    This class hooks into the forward and backward pass of the model's
    last convolutional layer to calculate feature map importance weights.
    """
    def __init__(self, model: torch.nn.Module, target_layer_name: str):
        self.model = model
        self.target_layer = self._find_target_layer(target_layer_name)
        self.feature_maps = {}
        self.gradients = {}
        self._register_hooks()

    def _find_target_layer(self, name: str):
        # Find the target layer in the EfficientNetV2 backbone
        modules = dict(self.model.backbone.named_modules())

        # First, try to use the provided name directly if it exists and is a Conv2d
        if name in modules and isinstance(modules[name], nn.Conv2d):
            print(f"Using specified target layer: '{name}'")
            return modules[name]

        # If the specified name is not found or not a Conv2d,
        # iterate to find the last Conv2d layer
        last_conv_layer = None
        last_conv_name = None
        for n, m in modules.items():
            if isinstance(m, nn.Conv2d):
                last_conv_layer = m
                last_conv_name = n

        if last_conv_layer is not None:
            print(f"Specified target layer '{name}' not found or not a Conv2d. Using last found Conv2d layer: '{last_conv_name}' as target.")
            return last_conv_layer
        else:
            raise ValueError(f"Could not find any Conv2d layer in the model's backbone for Grad-CAM. Named modules: {modules.keys()}")

    def _register_hooks(self):
        """Registers hooks to capture feature maps (forward) and gradients (backward)."""
        def forward_hook(module, input, output):
            # Store the feature map (output of the layer)
            self.feature_maps[0] = output.detach()

        def backward_hook(module, grad_in, grad_out):
            # Store the gradient (input to the layer, which is grad_out)
            self.gradients[0] = grad_out[0].detach()

        # Register the hooks on the identified target layer
        self.target_layer.register_forward_hook(forward_hook)
        self.target_layer.register_backward_hook(backward_hook)

    def __call__(self, input_tensor: torch.Tensor, target_category: int) -> np.ndarray:
        """
        Computes the Grad-CAM heatmap for a single input tensor.

        Args:
            input_tensor: The image tensor (e.g., [1, 3, 224, 224]).
            target_category: The index of the class to visualize (e.g., 0 for 'normal', 1 for 'fractured').

        Returns:
            A normalized and resized NumPy array representing the heatmap.
        """
        self.model.zero_grad()

        # 1. Forward pass
        output = self.model(input_tensor)

        # 2. Select the target score and perform a backward pass
        # The output must be a single item tensor for backward to work
        target_score = output[:, target_category]
        target_score.backward(retain_graph=True)

        # 3. Get the stored feature maps and gradients
        feature_maps = self.feature_maps[0]  # [B, C, H, W]
        gradients = self.gradients[0]      # [B, C, H, W]

        # We only process the first item in the batch (B=1)
        # 4. Calculate Alpha (Neuron Importance Weights)
        # Global Average Pooling (GAP) of the gradients across spatial dimensions (H, W)
        weights = torch.mean(gradients, dim=(2, 3), keepdim=True) # [1, C, 1, 1]

        # 5. Compute the weighted feature map
        # Sum of (alpha * feature_map) across the channel dimension (C)
        weighted_feature_map = weights * feature_maps
        cam = torch.sum(weighted_feature_map, dim=1) # [1, H, W]

        # 6. Apply ReLU to the CAM
        heatmap = F.relu(cam)

        # 7. Normalize and resize
        heatmap = heatmap.squeeze(0).cpu().numpy()
        heatmap = heatmap / np.max(heatmap) # Normalize to 0-1

        # Resize the heatmap to the original image size (224x224 from CFG)
        heatmap = cv2.resize(heatmap, (input_tensor.shape[2], input_tensor.shape[3]))

        return heatmap

SECTION 9: GRAD-CAM VISUALIZATION UTILITIES

In [15]:
# Utility functions for visualizing Grad-CAM heatmaps.
def visualize_grad_cam(original_image: np.ndarray, heatmap: np.ndarray, title: str, save_path=None):
    """
    Overlays the heatmap on the original image for visualization.

    Args:
        original_image: The original image as a NumPy array (H, W, 3).
        heatmap: The Grad-CAM heatmap (H, W) normalized between 0 and 1.
        title: The title for the plot.
        save_path: Optional path to save the visualization.
    """
    # Convert image to 0-255 range and 8-bit unsigned integer (for OpenCV color mapping)
    image_float = (original_image * 255).astype(np.uint8)

    # Apply colormap (e.g., JET)
    heatmap_jet = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
    heatmap_jet = cv2.cvtColor(heatmap_jet, cv2.COLOR_BGR2RGB) # Convert BGR to RGB

    # Superimpose the heatmap on the image
    # Alpha (0.4) controls the transparency of the heatmap
    superimposed_img = cv2.addWeighted(image_float, 0.6, heatmap_jet, 0.4, 0)

    # Plotting
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    ax[0].imshow(original_image)
    ax[0].set_title(f"Original: {title}", fontsize=10)
    ax[0].axis('off')

    ax[1].imshow(superimposed_img)
    ax[1].set_title(f"Grad-CAM: {title}", fontsize=10)
    ax[1].axis('off')

    plt.tight_layout()

    if save_path:
        plt.savefig(save_path)
    plt.show()

class GradCAMUtils:
    """Utility class for Grad-CAM operations."""

    @staticmethod
    def generate_and_save_gradcam_samples(model, dataloader, class_names, device, num_samples=5):
        """
        Generate Grad-CAM visualizations for random samples from the dataset.

        Args:
            model: Trained model for visualization
            dataloader: DataLoader containing the images
            class_names: List of class names
            device: Device to run computations on
            num_samples: Number of samples to visualize
        """
        print(f"\nGenerating {num_samples} Grad-CAM visualizations...")

        # Initialize GradCAM with the target layer
        # For EfficientNetV2, we'll use the last feature block
        grad_cam = GradCAM(model, target_layer_name='features.8')

        # Select random samples from the dataset
        sample_indices = np.random.choice(len(dataloader.dataset), num_samples, replace=False)

        for i, idx in enumerate(sample_indices):
            # Get the image and label
            image, label = dataloader.dataset[idx]
            image_tensor = image.unsqueeze(0).to(device)

            # Get model prediction
            model.eval()
            with torch.no_grad():
                logits = model(image_tensor)
                pred_prob = F.softmax(logits, dim=1)
                pred_label = torch.argmax(pred_prob).item()

            # Compute Grad-CAM for the predicted class
            heatmap = grad_cam(image_tensor, target_category=pred_label)

            # Prepare the image for visualization (convert from C, H, W to H, W, C)
            image_np = image_tensor.squeeze(0).cpu().numpy().transpose(1, 2, 0)

            # Denormalize the image
            mean = np.array([0.485, 0.456, 0.406])
            std = np.array([0.229, 0.224, 0.225])
            image_np = image_np * std + mean
            image_np = np.clip(image_np, 0, 1)

            # Create title
            title = (f"Pred: {class_names[pred_label]} ({pred_prob[0, pred_label].item():.2f}) "
                     f"| True: {class_names[label]}")

            # Visualize the Grad-CAM
            save_path = os.path.join(CFG.OUTPUT_DIR, f'gradcam_sample_{i}.png')
            visualize_grad_cam(image_np, heatmap, title, save_path)

        print(f"✅ Grad-CAM visualizations saved to {CFG.OUTPUT_DIR}")

print("✅ Grad-CAM visualization utilities defined.")

✅ Grad-CAM visualization utilities defined.


SECTION 10: ADVANCED TECHNIQUES

In [16]:
class AdvancedTechniques:
    """Class for implementing advanced techniques."""

    @staticmethod
    def k_fold_cross_validation(data_path, image_size, batch_size, k_folds=2):
        """Implement stratified K-fold cross-validation."""
        print(f"Starting {k_folds}-Fold Cross Validation")
        full_dataset = datasets.ImageFolder(
            os.path.join(data_path, 'train'),
            transform=DataTransforms.get_transforms(image_size, 'train')
        )
        labels = [label for _, label in full_dataset.samples]
        skf = StratifiedKFold(
            n_splits=k_folds,
            shuffle=True,
            random_state=CFG.SEED
        )
        fold_results = []

        for fold, (train_idx, val_idx) in enumerate(skf.split(np.zeros(len(labels)), labels)):
            print(f"\nFold {fold+1}/{k_folds}")
            train_subset = Subset(full_dataset, train_idx)
            val_subset = Subset(full_dataset, val_idx)

            train_loader = DataLoader(
                train_subset,
                batch_size=batch_size,
                shuffle=True,
                num_workers=2
            )
            val_loader = DataLoader(
                val_subset,
                batch_size=batch_size,
                shuffle=False,
                num_workers=2
            )

            model = HybridModel(
                cnn_model_name=CFG.MODEL_NAME,
                num_classes=CFG.NUM_CLASSES,
                d_model=CFG.D_MODEL,
                nhead=CFG.NHEAD,
                num_encoder_layers=CFG.NUM_ENCODER_LAYERS,
                dim_feedforward=CFG.DIM_FEEDFORWARD,
                dropout=CFG.DROPOUT
            ).to(CFG.device)

            ModelUtils.initialize_weights(model)
            criterion = ModelUtils.get_loss_function()
            optimizer = ModelUtils.get_optimizer(model, CFG.LEARNING_RATE)
            scheduler = ModelUtils.get_lr_scheduler(optimizer, epochs=CFG.EPOCHS)

            best_val_acc = 0.0
            for epoch in range(CFG.EPOCHS):
                train_loss, train_acc = TrainingUtils.train_one_epoch(
                    model, train_loader, criterion, optimizer, CFG.device
                )
                val_loss, val_acc, _, _, _ = TrainingUtils.evaluate(
                    model, val_loader, criterion, CFG.device, split='Validation'
                )
                scheduler.step()

                if val_acc > best_val_acc:
                    best_val_acc = val_acc
                    torch.save(
                        model.state_dict(),
                        os.path.join(CFG.OUTPUT_DIR, f'fold_{fold+1}_best_model.pth')
                    )

            fold_results.append({'fold': fold+1, 'best_val_acc': best_val_acc})
            print(f"Fold {fold+1} completed. Best validation accuracy: {best_val_acc:.4f}")

        avg_val_acc = np.mean([result['best_val_acc'] for result in fold_results])
        std_val_acc = np.std([result['best_val_acc'] for result in fold_results])
        print(f"\nCross-Validation Results\nAverage validation accuracy: {avg_val_acc:.4f} ± {std_val_acc:.4f}")

        results_df = pd.DataFrame(fold_results)
        results_df.to_csv(os.path.join(CFG.OUTPUT_DIR, 'kfold_results.csv'), index=False)

        return fold_results

    @staticmethod
    def plot_kfold_results(fold_results):
        """Visualizes K-Fold Cross-Validation results with a bar chart."""
        print("\nGenerating K-Fold visualization")
        folds = [f"Fold {result['fold']}" for result in fold_results]
        accuracies = [result['best_val_acc'] for result in fold_results]

        avg_acc = np.mean(accuracies)
        std_acc = np.std(accuracies)

        plt.figure(figsize=(10, 6))
        bars = plt.bar(folds, accuracies, color='skyblue', alpha=0.7, yerr=std_acc, capsize=5)
        plt.axhline(
            y=avg_acc,
            color='red',
            linestyle='--',
            label=f'Average Accuracy: {avg_acc:.4f} ± {std_acc:.4f}'
        )

        for bar, acc in zip(bars, accuracies):
            plt.text(
                bar.get_x() + bar.get_width()/2,
                bar.get_height() + std_acc + 0.002,
                f'{acc:.4f}',
                ha='center',
                va='bottom',
                fontsize=10
            )

        plt.xlabel('Cross-Validation Fold', fontsize=12)
        plt.ylabel('Best Validation Accuracy', fontsize=12)
        plt.title('K-Fold Cross-Validation Performance', fontsize=16, fontweight='bold')
        plt.ylim(0, 1)
        plt.legend(fontsize=11)
        plt.grid(True, linestyle='--', alpha=0.6)
        plt.tight_layout()
        plt.savefig(os.path.join(CFG.OUTPUT_DIR, 'kfold_results_plot.png'))
        plt.show()

    @staticmethod
    def objective(trial, train_loader, val_loader):
        """Objective function for Optuna hyperparameter tuning."""
        lr = trial.suggest_float('lr', 1e-5, 1e-3, log=True)
        weight_decay = trial.suggest_float('weight_decay', 1e-6, 1e-3, log=True)
        dropout = trial.suggest_float('dropout', 0.1, 0.5)
        nhead = trial.suggest_categorical('nhead', [4, 8, 16])

        model = HybridModel(
            cnn_model_name=CFG.MODEL_NAME,
            num_classes=CFG.NUM_CLASSES,
            d_model=CFG.D_MODEL,
            nhead=nhead,
            num_encoder_layers=CFG.NUM_ENCODER_LAYERS,
            dim_feedforward=CFG.DIM_FEEDFORWARD,
            dropout=dropout
        ).to(CFG.device)

        criterion = ModelUtils.get_loss_function()
        optimizer = ModelUtils.get_optimizer(model, lr, weight_decay)

        best_val_acc = 0.0
        for epoch in range(2):
            train_loss, train_acc = TrainingUtils.train_one_epoch(
                model, train_loader, criterion, optimizer, CFG.device
            )
            val_loss, val_acc, _, _, _ = TrainingUtils.evaluate(
                model, val_loader, criterion, CFG.device, split='Validation'
            )
            if val_acc > best_val_acc:
                best_val_acc = val_acc

        return best_val_acc

    @staticmethod
    def hyperparameter_tuning(train_loader, val_loader, n_trials=2):
        """Perform hyperparameter tuning using Optuna."""
        print(f"Starting hyperparameter tuning with {n_trials} trials")
        study = optuna.create_study(direction='maximize')
        study.optimize(
            lambda trial: AdvancedTechniques.objective(trial, train_loader, val_loader),
            n_trials=n_trials
        )

        best_params = study.best_params
        best_value = study.best_value

        print(f"Best parameters: {best_params}\nBest validation accuracy: {best_value:.4f}")

        with open(os.path.join(CFG.OUTPUT_DIR, 'best_hyperparams.json'), 'w') as f:
            json.dump(best_params, f, indent=2)

        return best_params

    @staticmethod
    def evaluate_model_calibration(model, dataloader, device):
        """Compute ECE and plot a reliability diagram."""
        model.eval()
        calibrator = CalibrationError(
            task="multiclass",
            num_classes=CFG.NUM_CLASSES
        ).to(device)

        all_preds, all_labels, all_probs = [], [], []

        with torch.no_grad():
            for inputs, labels in dataloader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                probs = torch.softmax(outputs, dim=1)
                _, preds = torch.max(outputs, 1)

                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
                all_probs.extend(probs.cpu().numpy())

        all_preds = torch.tensor(all_preds).to(device)
        all_labels = torch.tensor(all_labels).to(device)
        all_probs = torch.tensor(all_probs).to(device)

        # No accuracy cap or label flipping here
        ece = calibrator(all_probs, all_labels)

        plt.figure(figsize=(8, 6))
        plt.hist2d(
            all_probs[:, 1].cpu().numpy(),
            (all_preds == 1).cpu().numpy(),
            bins=10,
            range=[[0, 1], [0, 1]],
            cmap='Blues'
        )
        plt.plot([0, 1], [0, 1], 'r--')
        plt.xlabel('Mean predicted probability')
        plt.ylabel('Fraction of positives')
        plt.title(f'Reliability Diagram (ECE: {ece.item():.4f})')
        plt.savefig(os.path.join(CFG.OUTPUT_DIR, 'reliability_diagram.png'))
        plt.show()

        return ece.item()

print("✅ Advanced techniques functions defined.")


✅ Advanced techniques functions defined.


In [19]:
# Simple ANSI color helper for pretty console output
class C:
    HEADER = "\033[95m"
    BLUE = "\033[94m"
    CYAN = "\033[96m"
    GREEN = "\033[92m"
    YELLOW = "\033[93m"
    RED = "\033[91m"
    BOLD = "\033[1m"
    DIM = "\033[2m"
    RESET = "\033[0m"


SECTION 11: MAIN EXECUTION PIPELINE

In [ ]:
# The main function that orchestrates the entire training and evaluation process.
def main():
    """Main execution function."""
    print(f"{C.CYAN}{C.BOLD}🚀 Starting Bone Fracture Detection Pipeline{C.RESET}")

    # Phase 1: Data Preparation
    print(f"\n{C.HEADER}{C.BOLD}=== Phase 1: Data Preparation ==={C.RESET}")
    data_path = DatasetManager.load_dataset(CFG.DATA_PATH)
    image_datasets = DatasetManager.create_dataset_splits(data_path, CFG.IMAGE_SIZE)
    dataloaders, dataset_sizes, class_names = DatasetManager.create_dataloaders(
        image_datasets, CFG.BATCH_SIZE
    )
    metadata = DatasetManager.prepare_metadata(image_datasets)
    print(f"{C.GREEN}✅ Data prepared. Splits -> "
          f"Train: {dataset_sizes['train']}, "
          f"Val: {dataset_sizes['val']}, "
          f"Test: {dataset_sizes['test']}{C.RESET}")
    print(f"{C.DIM}Classes: {class_names}{C.RESET}")

    # Phase 2: Model Construction
    print(f"\n{C.HEADER}{C.BOLD}=== Phase 2: Model Construction ==={C.RESET}")
    model = HybridModel(
        cnn_model_name=CFG.MODEL_NAME,
        num_classes=CFG.NUM_CLASSES,
        d_model=CFG.D_MODEL,
        nhead=CFG.NHEAD,
        num_encoder_layers=CFG.NUM_ENCODER_LAYERS,
        dim_feedforward=CFG.DIM_FEEDFORWARD,
        dropout=CFG.DROPOUT
    ).to(CFG.device)
    ModelUtils.initialize_weights(model)
    ModelUtils.freeze_backbone(model)
    print(f"{C.GREEN}✅ Model initialized on device: {CFG.device}{C.RESET}")
    print(f"{C.DIM}Backbone: {CFG.MODEL_NAME}, Transformer d_model={CFG.D_MODEL}{C.RESET}")

    # Phase 3: Training Pipeline
    print(f"\n{C.HEADER}{C.BOLD}=== Phase 3: Training Pipeline ==={C.RESET}")
    criterion = ModelUtils.get_loss_function()
    optimizer = ModelUtils.get_optimizer(model, CFG.LEARNING_RATE)
    scheduler = ModelUtils.get_lr_scheduler(optimizer, epochs=CFG.EPOCHS)

    history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': []
    }
    best_val_acc = 0.0

    # For patience-based early stopping
    best_val_loss = float('inf')
    patience_counter = 0

    for epoch in range(CFG.EPOCHS):
        print(
            f"\n{C.BLUE}{C.BOLD}Epoch {epoch + 1}/{CFG.EPOCHS}"
            f"{C.RESET}\n{C.DIM}" + "-" * 40 + f"{C.RESET}"
        )

        # Unfreeze backbone halfway through training and reduce LR
        if epoch == CFG.EPOCHS // 2:
            print(f"{C.YELLOW}🔓 Unfreezing backbone and lowering learning rate"
                  f" to {CFG.LEARNING_RATE * 0.1:.1e}{C.RESET}")
            ModelUtils.unfreeze_backbone(model)
            optimizer = ModelUtils.get_optimizer(model, CFG.LEARNING_RATE * 0.1)
            scheduler = ModelUtils.get_lr_scheduler(
                optimizer, epochs=CFG.EPOCHS - epoch
            )

        train_loss, train_acc = TrainingUtils.train_one_epoch(
            model, dataloaders['train'], criterion, optimizer, CFG.device
        )
        val_loss, val_acc, _, _, _ = TrainingUtils.evaluate(
            model, dataloaders['val'], criterion, CFG.device
        )

        scheduler.step()

        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)

        print(
            f"{C.CYAN}📊 Epoch summary -> "
            f"Train Loss: {train_loss:.4f}, "
            f"Train Acc: {train_acc:.4f}, "
            f"Val Loss: {val_loss:.4f}, "
            f"Val Acc: {val_acc:.4f}{C.RESET}"
        )

        # Save best model by validation accuracy
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            ModelUtils.save_checkpoint(
                model,
                optimizer,
                epoch,
                val_acc,
                os.path.join(CFG.OUTPUT_DIR, 'best_model.pth')
            )
            print(
                f"{C.GREEN}💾 New best model saved with "
                f"Val Acc: {val_acc:.4f}{C.RESET}"
            )

        # Patience-based early stopping on validation loss
        if val_loss < best_val_loss - 1e-4:  # small delta to avoid noise
            best_val_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1
            print(
                f"{C.YELLOW}⏳ Early stopping patience: "
                f"{patience_counter}/{CFG.EARLY_STOPPING_PATIENCE}{C.RESET}"
            )
            if patience_counter >= CFG.EARLY_STOPPING_PATIENCE:
                print(
                    f"{C.RED}{C.BOLD}🛑 Early stopping triggered "
                    f"(no improvement in validation loss).{C.RESET}"
                )
                break

    # Phase 4: Model Evaluation
    print(f"\n{C.HEADER}{C.BOLD}=== Phase 4: Model Evaluation ==={C.RESET}")
    best_model = HybridModel(
        cnn_model_name=CFG.MODEL_NAME,
        num_classes=CFG.NUM_CLASSES,
        d_model=CFG.D_MODEL,
        nhead=CFG.NHEAD,
        num_encoder_layers=CFG.NUM_ENCODER_LAYERS,
        dim_feedforward=CFG.DIM_FEEDFORWARD,
        dropout=CFG.DROPOUT
    ).to(CFG.device)
    best_model = ModelUtils.load_best_model(
        best_model,
        os.path.join(CFG.OUTPUT_DIR, 'best_model.pth')
    )

    test_loss, test_acc, test_labels, test_preds, test_probs = TrainingUtils.test_inference(
        best_model, dataloaders['test'], criterion, CFG.device
    )
    test_metrics = MetricsAndVisualization.compute_metrics(
        test_labels, test_preds, test_probs
    )

    print(f"\n{C.CYAN}{C.BOLD}📈 Test set performance:{C.RESET}")
    print(
        f"{C.GREEN}"
        f"  • Accuracy : {test_metrics['accuracy']:.4f}\n"
        f"  • Precision: {test_metrics['precision']:.4f}\n"
        f"  • Recall   : {test_metrics['recall']:.4f}\n"
        f"  • F1-score : {test_metrics['f1']:.4f}\n"
        f"  • AUC      : {test_metrics['auc']:.4f}"
        f"{C.RESET}"
    )

    MetricsAndVisualization.plot_curves(history, test_acc, test_metrics['auc'])
    MetricsAndVisualization.plot_confusion_matrix(test_labels, test_preds, class_names)
    MetricsAndVisualization.analyze_predictions(
        best_model, dataloaders['test'], class_names, CFG.device
    )

    # Phase 4.5: Grad-CAM Visualization
    print(f"\n{C.HEADER}{C.BOLD}=== Phase 4.5: Grad-CAM Visualization ==={C.RESET}")
    GradCAMUtils.generate_and_save_gradcam_samples(
        best_model,
        dataloaders['test'],
        class_names,
        CFG.device,
        num_samples=5
    )
    print(f"{C.GREEN}✅ Grad-CAM samples generated and saved.{C.RESET}")

    # Phase 5: Robustness & Optimization
    print(f"\n{C.HEADER}{C.BOLD}=== Phase 5: Robustness & Optimization ==={C.RESET}")
    print(f"{C.CYAN}Starting K-Fold Cross-Validation...{C.RESET}")
    k_fold_results = AdvancedTechniques.k_fold_cross_validation(
        CFG.DATA_PATH, CFG.IMAGE_SIZE, CFG.BATCH_SIZE
    )
    AdvancedTechniques.plot_kfold_results(k_fold_results)

    print(f"\n{C.CYAN}Starting hyperparameter tuning...{C.RESET}")
    best_params = AdvancedTechniques.hyperparameter_tuning(
        dataloaders['train'], dataloaders['val']
    )
    print(f"{C.GREEN}✅ Best hyperparameters found: {best_params}{C.RESET}")

    ece = AdvancedTechniques.evaluate_model_calibration(
        best_model, dataloaders['test'], CFG.device
    )
    print(f"{C.CYAN}Calibration ECE: {ece:.4f}{C.RESET}")

    print(f"\n{C.GREEN}{C.BOLD}🎉 Pipeline completed successfully!{C.RESET}")


if __name__ == "__main__":
    main()


🚀 Starting Bone Fracture Detection Pipeline

=== Phase 1: Data Preparation ===
Loading dataset from /content/drive/MyDrive/SaucyDataset
Datasets loaded. Class names: ['fracture', 'normal']
   - Train set size: 1701 images
   - Val set size: 212 images
   - Test set size: 214 images
Metadata prepared and saved to /content/drive/MyDrive/SaucyDataset_Output/metadata.json
✅ Data prepared. Splits -> Train: 1701, Val: 212, Test: 214
Classes: ['fracture', 'normal']

=== Phase 2: Model Construction ===


model.safetensors:   0%|          | 0.00/86.5M [00:00<?, ?B/s]

Weights initialized
Backbone frozen
✅ Model initialized on device: cuda:0
Backbone: tf_efficientnetv2_s.in1k, Transformer d_model=256

=== Phase 3: Training Pipeline ===

Epoch 1/30
----------------------------------------


Validation: 100%|██████████| 27/27 [00:27<00:00,  1.03s/batch]


📊 Epoch summary -> Train Loss: 0.2625, Train Acc: 0.9353, Val Loss: 0.2289, Val Acc: 0.9434
Checkpoint saved to /content/drive/MyDrive/SaucyDataset_Output/best_model.pth
💾 New best model saved with Val Acc: 0.9434

Epoch 2/30
----------------------------------------


...

Validation: 100%|██████████| 27/27 [00:03<00:00,  8.36batch/s]


📊 Epoch summary -> Train Loss: 0.1088, Train Acc: 0.9600, Val Loss: 0.2681, Val Acc: 0.8679
⏳ Early stopping patience: 3/5

Epoch 9/30
----------------------------------------


Validation: 100%|██████████| 27/27 [00:02<00:00,  9.18batch/s]


📊 Epoch summary -> Train Loss: 0.0991, Train Acc: 0.9594, Val Loss: 0.1540, Val Acc: 0.9575
⏳ Early stopping patience: 4/5

Epoch 10/30
----------------------------------------


Training:  34%|███▍      | 73/213 [00:10<00:26,  5.25batch/s, acc=0.9692, loss=0.0748]